# Funções Globais

In [3]:
import numpy as np
import sympy as sp
from tqdm import tqdm
import plotly.graph_objects as go

In [4]:
def gradiente_simbolico(xs_simbolicos, objetivo_simbolico):
    print(xs_simbolicos)

    return sp.Array([objetivo_simbolico.diff(x) for x in xs_simbolicos])

def funcao_gradiente(variaveis_simbolicas, gradiente_simbolico):
    return sp.lambdify(variaveis_simbolicas, gradiente_simbolico, 'numpy')

def funcao_hessiano(variaveis_simbolicas, gradiente_simbolico):
    hessiano_simbolico = sp.hessian(gradiente_simbolico, variaveis_simbolicas)
    return sp.lambdify(variaveis_simbolicas, hessiano_simbolico, 'numpy')

In [7]:
def metodo_newton(funcao_gradiente_numerico, funcao_hessiano_numerico, x, c, l=None, s=None, eps=0.001, max_iter=10000):
    x_copia = x.copy()
    variaveis_extra = [c]

    if l is not None and l.any():
        variaveis_extra.append(l)
    if s is not None and s.any():
        variaveis_extra.append(s)

    for _ in range(max_iter):
        gradiente = funcao_gradiente_numerico(*x_copia, *variaveis_extra) 

        hessiano = funcao_hessiano_numerico(*x_copia, *variaveis_extra)

        gradiente = gradiente.flatten()

        if np.linalg.norm(gradiente) < eps:
            break

        delta = np.linalg.solve(hessiano, gradiente)
        x_copia -= delta

    return x_copia

def metodo_gradiente(funcao_gradiente_numerico, x, c, l=None, s=None, alpha=0.001, eps=0.001, max_iter=10000):
    x_copia = x.copy()

    variaveis_extra = [c]

    if l is not None and len(l):
        variaveis_extra.append(l)
    if s is not None and len(s):
        variaveis_extra.append(s)
    
    for _ in range(max_iter):

        gradiente = funcao_gradiente_numerico(*x_copia, *variaveis_extra) 

        s_novo = None
        
        grad_x = gradiente

        if s is not None and len(s):
            grad_x = gradiente[:len(x)].astype("float64")
            s_novo = gradiente[len(x):]

        if np.linalg.norm(grad_x) < eps:
            break

        x_copia -= alpha * grad_x

    return x_copia, s_novo

def penalidade(tipo, objetivo_simbolico, xs_simbolicos, h, g, ponto_inicial, fator, eps=0.01, max_iter=10000):
    
    x = np.array(ponto_inicial).astype("float64")
    lista_x = [x]
    l_numerico = None
    c_numerico = None
    psi = None
    penalizacao_simbolico = None

    c = sp.symbols('c')
    l_simbolico = sp.MatrixSymbol("l", len(h) + len(g), 1)
    l = sp.Matrix(l_simbolico)

    s_simbolico = sp.MatrixSymbol("s", len(g), 1)
    s = sp.Matrix(s_simbolico)
    s_numerico = np.array([2]*len(g)).astype("float64")

    if tipo in ["externa", "quadratica", "lagrangiana"]:
        c_numerico = 0.1

        if tipo == "externa":
            psi = (h.dot(h) if len(h) else 0) + sum(
                sp.Max(0, restricao) ** 2 for restricao in g
            )
            penalizacao_simbolico = objetivo_simbolico + c * psi

        else:
            folga_quadrado = s.applyfunc(lambda x: x**2)

            g_transformado = g + folga_quadrado
            novo_h = sp.Matrix([*h, *g_transformado])

            if tipo == "quadratica":
                psi = novo_h.dot(novo_h)
                penalizacao_simbolico = objetivo_simbolico + c/2 * psi
            else:
                penalizacao_simbolico = objetivo_simbolico + l.dot(novo_h) + c/2 * psi
                l_numerico = np.array([1]*len(h)).astype("float64")

    elif tipo == "interna":
        c_numerico = 256

        # Barreira logaritmica
        psi = -sum(sp.log(-gx) for gx in g)

        penalizacao_simbolico = objetivo_simbolico + c * psi
    
    gradiente_sym = gradiente_simbolico(xs_simbolicos, penalizacao_simbolico)

    funcao_grad_numerico = funcao_gradiente([*xs_simbolicos, c], gradiente_sym)
    funcao_hessiano_numerico = funcao_hessiano([*xs_simbolicos, c], gradiente_sym)

    if tipo in ["quadratica", "lagrangiana"]:
        gradiente_sym = gradiente_simbolico([*xs_simbolicos, *s], penalizacao_simbolico)

        funcao_grad_numerico = funcao_gradiente([*xs_simbolicos, c, s], gradiente_sym)
        funcao_hessiano_numerico = funcao_hessiano([*xs_simbolicos, c, s], gradiente_sym)

    if tipo == "lagrangiana":
        funcao_grad_numerico = funcao_gradiente([*xs_simbolicos, c, l, s], gradiente_sym)
        funcao_hessiano_numerico = funcao_hessiano([*xs_simbolicos, c, s], gradiente_sym)
        
    print()
    print(gradiente_sym)
    print()
    for _ in tqdm(range(max_iter)):
        print("x", x)
        print("s", s_numerico)
        print("c", c_numerico)

        if tipo in ["lagrangiana", "quadratica"]:
            novo_x, s_numerico = metodo_gradiente(funcao_grad_numerico, x, c_numerico, l_numerico, s_numerico)

        else:
            novo_x, s_numerico = metodo_gradiente(funcao_grad_numerico, x, c_numerico, l_numerico, s_numerico)

        lista_x.append(novo_x)

        if np.linalg.norm(x - novo_x) < eps:
            break

        x = novo_x.copy()

        if tipo in ["externa", "quadratica", "lagrangiana"]:
            c_numerico *= fator
        elif tipo == "interna":
            c_numerico /= fator

        if tipo == "lagrangiana":
            psi_funcao = sp.lambdify(xs_simbolicos, h, "numpy")
            psi_valor = psi_funcao(*x)
            psi_dimensao_reduzida = psi_valor.flatten()

            l_numerico += c_numerico*psi_dimensao_reduzida

    return x, np.array(lista_x)

def plotar_funcao_objetivo(f, z_func, xs, nome):
    x_range = (-10, 10)
    y_range = (-10, 10)
    # Define the space for x1 and x2

    x1 = np.linspace(*x_range, 100)
    x2 = np.linspace(*y_range, 100)
    x1, x2 = np.meshgrid(x1, x2)
    x3 = z_func(*xs)  # apply the constraint

    # Calculate the values of the function for the grid of points
    Z = f(*xs)

    # Separate the points
    xs_first = xs[0]
    xs_last = xs[-1]
    xs_middle = xs[1:-1]

    Z_points_first = f(*xs_first)
    Z_points_last = f(*xs_last)
    Z_points_middle = np.array([f(*x) for x in xs_middle])

    # Create the plot
    fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                          go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                       mode='markers', marker=dict(size=6, color='green')),
                          go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                       mode='markers', marker=dict(size=6, color='red')),
                          go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                       mode='markers', marker=dict(size=10, color='blue'))])
    fig.update_layout(scene=dict(
                        xaxis_title='X1',
                        yaxis_title='X2',
                        zaxis_title='Z'),
                      title_text="Função Objetivo 3D Interativa")

    fig.write_html(f"{nome}.html")

    fig.show()


In [11]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-1, -2, 4], [-2, 0.5, 3], [-0.5, -3, 1], [-1, 5, 3], [2, -3, 1]])
b_simbolico = sp.Matrix([-2, 0, 3, -1, 7])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([x1 + x2 + x3 - 1])
g = sp.Matrix([-x1, -x2, -x3])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [10, -1, 10]

x_ponto, x_lista = penalidade("quadratica", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

TypeError: unsupported operand type(s) for *: 'Mul' and 'NoneType'

# 1A - OK

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-2, 0, 0], [0, np.sqrt(5), 0], [1, 0, 0], [0, 1, 1], [-1, 0, -1]])
b_simbolico = sp.Matrix([7, 2, 7, -2, 5])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([x1 + x2 + x3 - 1])
g = sp.Matrix([[-x1], [-x2], [-x3]])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [1, 1, 1]

x_ponto, x_lista = penalidade("externa", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)


In [ ]:
# Função objetivo
def f(x1, x2, x3):
    return objetivo(x1, x2, x3)

# Defina o espaço para x1, x2 e x3
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)
x3 = 1 - x1 - x2  # de acordo com a restrição

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2, x3)

xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("1a.html")

fig.show()


# 1B - OK

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-2, 0, 0], [0, np.sqrt(5), 0], [1, 0, 0], [0, 1, 1], [-1, 0, -1]])
b_simbolico = sp.Matrix([7, 2, 7, -2, 5])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([])
g = sp.Matrix([[-x1], [-x2], [-x3], [-x1 - x2 - x3 + 1], [x1 + x2 + x3 - 1]])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [1, 1, 1]
x_ponto, x_lista = penalidade("interna", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
# Função objetivo
def f(x1, x2, x3):
    return objetivo(x1, x2, x3)

# Defina o espaço para x1, x2 e x3
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)
x3 = 1 - x1 - x2  # de acordo com a restrição

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2, x3)

xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("1b.html")

fig.show()

# 2A - OK

![config](imgs/config.png)

![img1](imgs/p1.png)

![img2](imgs/p2.png)

![img3](imgs/p3.png)

# 2B - OK

In [ ]:
# Defina as variáveis simbólicas
x1, x2 = sp.symbols('x1 x2')
xs_simbolicos = sp.Array([x1, x2])

objetivo_simbolico = 2*x1 + 3*x2 + 4*x1**2 + 2*x1*x2 + x2**2
objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

h = sp.Matrix([])

g1x = -(x1 - x2)
g2x = x1 + x2 - 4
g3x = -(x1 - 3)
g = sp.Matrix([g1x, g2x, g3x])

x0 = [5, 5]

x_ponto, x_lista = penalidade("externa", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("2b.html")

fig.show()


# 2C - OK

In [ ]:
# Utilizar pontos que respeitam as restrições
x0 = [3.5, 0.2]

x_ponto, x_lista = penalidade("interna", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("2c.html")

fig.show()

# 3A

In [ ]:
# Define a função objetivo
def objective(x):
    x1, x2, x3 = x
    return x1**3 - 6*x1**2 + 11*x1 + x3

# Define as restrições
def constraint1(x):
    x1, x2, x3 = x
    return x1**2 + x2**2 - x3**2

def constraint2(x):
    x1, x2, x3 = x
    return 4-(x1**2 + x2**2 + x3**2)

def constraint3(x):
    _, _, x3 = x
    return 5 - x3

def constraint4(x):
    x1, _, _ = x
    return -x1
    
def constraint5(x):
    _, x2, x3 = x
    return -(x2*x3)


# Define as restrições no formato para o Scipy
constraints = [{'type': 'ineq', 'fun': constraint1},
               {'type': 'ineq', 'fun': constraint2},
               {'type': 'ineq', 'fun': constraint3},
               {'type': 'ineq', 'fun': constraint4},
               {'type': 'ineq', 'fun': constraint5}]

# Define o ponto inicial
x0 = np.array([0,0,0])

# Chama a função minimize
solution = minimize(objective, x0, method='SLSQP', constraints=constraints)

# Printa a solução
print(solution)

# 3B

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3, c = sp.symbols('x1 x2 x3 c')
variaveis_simbolicas = sp.Array([x1, x2, x3])
xs_simbolicos = variaveis_simbolicas[:-1]

objetivo_simbolico = x1**3 - 6*x1**2 + 11*x1 + x3
objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
g1x = x1**2 + x2**2 - x3**2
g2x = -(x1**2 + x2**2 + x3**2 - 4)
g3x = x3 - 5
g4x = -x1
g5x = -x2
g6x = -x3

g = sp.Matrix([g1x, g2x, g3x, g4x, g5x, g6x])

x0 = [7, 7, 4]

x_ponto, x_lista = penalidade("externa", objetivo_simbolico, variaveis_simbolicas, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo para uma variável fixa
def f(x1, x2):
    return objetivo(x1, x2, x_ponto[2]) # x_ponto[2] é o valor final de x3 após a otimização

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe os pontos
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first[:2])  # pegamos apenas as duas primeiras coordenadas
Z_points_last = f(*xs_last[:2])
Z_points_middle = np.array([f(*x[:2]) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("3b.html")

fig.show()

# 3C

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3, c = sp.symbols('x1 x2 x3 c')
variaveis_simbolicas = sp.Array([x1, x2, x3])
xs_simbolicos = variaveis_simbolicas[:-1]

objetivo_simbolico = x1**3 - 6*x1**2 + 11*x1 + x3
objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
g1x = x1**2 + x2**2 - x3**2
g2x = -(x1**2 + x2**2 + x3**2 - 4)
g3x = x3 - 5
g4x = -x1
g5x = -x2
g6x = -x3

g = sp.Matrix([g1x, g2x, g3x, g4x, g5x, g6x])

x0 = [-1, -1, -1]

x_ponto, x_lista = penalidade("interna", objetivo_simbolico, variaveis_simbolicas, h, g, x0, 2)

# 4A - OK

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define a função objetivo
def objective(x):
    x1, x2, x3 = x
    return (x1-x2)**2 + (x2-x3)**4
    
# Define as restrições
def constraint1(x):
    x1, x2, x3 = x
    return x1 + x1*x2**2 + x3**4 - 3


# Define as restrições no formato para o Scipy
constraints = [{'type': 'eq', 'fun': constraint1}]
# Define o ponto inicial
x0 = np.array([0,0,0])

# Chama a função minimize
solution = minimize(objective, x0, method='SLSQP', constraints=constraints)

# Printa a solução
print(solution.x)

# 4B - OK

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

objetivo_simbolico = (x1-x2)**2 + (x2-x3)**4
objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

h = sp.Matrix([x1 + x1*x2**2 + x3**4 - 3])
g = sp.Matrix([])

x0 = [0, 0, 0]

x_ponto, x_lista = penalidade("externa", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo para uma variável fixa
def f(x1, x2):
    return objetivo(x1, x2, x_ponto[2]) # x_ponto[2] é o valor final de x3 após a otimização

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe os pontos
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first[:2])  # pegamos apenas as duas primeiras coordenadas
Z_points_last = f(*xs_last[:2])
Z_points_middle = np.array([f(*x[:2]) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("4b.html")

fig.show()


# 4C - OK

In [ ]:
x0 = [3, 0, 0]
x_ponto, x_lista = penalidade("interna", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
# import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2, x3):
    return objetivo(x1, x2, x3)

# Defina o espaço para x1, x2 e x3
x1 = np.linspace(0, 1, 100)
x2 = np.linspace(0, 1, 100)
x1, x2 = np.meshgrid(x1, x2)
x3 = (3 - x1 - x1*x2**2)**(1/4)  # de acordo com a restrição

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2, x3)

xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("4c.html")

fig.show()

# 5A - OK

In [ ]:
# Defina as variáveis simbólicas
x1, x2 = sp.symbols('x1 x2')
xs_simbolicos = sp.Array([x1, x2])

objetivo_simbolico = x1 + x2

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrição de igualdade
h = sp.Matrix([x1**2 + x2**2 - 2])
g = sp.Matrix([])

x0 = [0, 0]

x_ponto, x_lista = penalidade("quadratica", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("5_a.html")

fig.show()

# 5B - OK

In [ ]:
# Defina as variáveis simbólicas
x1, x2 = sp.symbols('x1 x2')
xs_simbolicos = sp.Array([x1, x2])

objetivo_simbolico = x1 + x2

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrição de igualdade
h = sp.Matrix([x1**2 + x2**2 - 2])
g = sp.Matrix([])

x0 = [0, 0]

x_ponto, x_lista = penalidade("lagrangiana", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo
def f(x1, x2):
    return objetivo(x1, x2)

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe o último ponto
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first)
Z_points_last = f(*xs_last)
Z_points_middle = np.array([f(*x) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("5b.html")

fig.show()


# 6A

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo para uma variável fixa
def f(x1, x2):
    return objetivo(x1, x2, x_ponto[2]) # x_ponto[2] é o valor final de x3 após a otimização

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe os pontos
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first[:2])  # pegamos apenas as duas primeiras coordenadas
Z_points_last = f(*xs_last[:2])
Z_points_middle = np.array([f(*x[:2]) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("6a.html")

fig.show()

# 6B

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-1, -2, 4], [-2, 0.5, 3], [-0.5, -3, 1], [-1, 5, 3], [2, -3, 1]])
b_simbolico = sp.Matrix([-2, 0, 3, -1, 7])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([x1 + x2 + x3 - 1, -x1, -x2, -x3])
g = sp.Matrix([])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [10, -1, 10]

x_ponto, x_lista = penalidade("lagrangiana", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo para uma variável fixa
def f(x1, x2):
    return objetivo(x1, x2, x_ponto[2]) # x_ponto[2] é o valor final de x3 após a otimização

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe os pontos
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first[:2])  # pegamos apenas as duas primeiras coordenadas
Z_points_last = f(*xs_last[:2])
Z_points_middle = np.array([f(*x[:2]) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("6b.html")

fig.show()

# 7A

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[3, -7, 2], [0, -4, 6], [1, -1, 4], [1, -2, 1], [1, -2, 4]])
b_simbolico = sp.Matrix([-2, 3, 4, -1, 0])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([x1 + x2 + x3 - 1])
g = sp.Matrix([[-x1], [-x2], [-x3]])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [10, -1, 10]

x_ponto, x_lista = penalidade("quadratica", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Função objetivo para uma variável fixa
def f(x1, x2):
    return objetivo(x1, x2, x_ponto[2]) # x_ponto[2] é o valor final de x3 após a otimização

# Defina o espaço para x1 e x2
x1 = np.linspace(-10, 10, 100)
x2 = np.linspace(-10, 10, 100)
x1, x2 = np.meshgrid(x1, x2)

# Calcule os valores da função para a grade de pontos
Z = f(x1, x2)

# Separe os pontos
xs_first = x_lista[0]
xs_last = x_lista[-1]
xs_middle = x_lista[1:-1]

Z_points_first = f(*xs_first[:2])  # pegamos apenas as duas primeiras coordenadas
Z_points_last = f(*xs_last[:2])
Z_points_middle = np.array([f(*x[:2]) for x in xs_middle])

# Crie o gráfico
fig = go.Figure(data=[go.Surface(z=Z, x=x1, y=x2, colorscale='Viridis', opacity=0.8),
                      go.Scatter3d(x=[xs_first[0]], y=[xs_first[1]], z=[Z_points_first], 
                                   mode='markers', marker=dict(size=6, color='green')),
                      go.Scatter3d(x=xs_middle[:,0], y=xs_middle[:,1], z=Z_points_middle, 
                                   mode='markers', marker=dict(size=6, color='red')),
                      go.Scatter3d(x=[xs_last[0]], y=[xs_last[1]], z=[Z_points_last], 
                                   mode='markers', marker=dict(size=10, color='blue'))])
fig.update_layout(scene=dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Z'),
                  title_text="Função Objetivo 3D Interativa")

fig.write_html("7a.html")

fig.show()


# 7B

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3 = sp.symbols('x1 x2 x3')
xs_simbolicos = sp.Array([x1, x2, x3])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[3, -7, 2], [0, -4, 6], [1, -1, 4], [1, -2, 1], [1, -2, 4]])
b_simbolico = sp.Matrix([-2, 3, 4, -1, 0])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([x1 + x2 + x3 - 1])
g = sp.Matrix([[-x1], [-x2], [-x3]])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [10, -1, 10]

x_ponto, x_lista = penalidade("lagrangiana", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

# 8A

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')
xs_simbolicos = sp.Array([x1, x2, x3, x4])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-5, 7, -1, 1], [-1, -1, 1, 0], [0, 4, -6, 3], [0, 0, 0, 8], [-1, 1, 1, 0], [0, 2, 3, 0], [0, 1, -3, 3]])
b_simbolico = sp.Matrix([0, -1, -5, 3, 2, 0, 2])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([x1 + x2 + x3 + x4 - 1])
g = sp.Matrix([[-x1], [-x2], [-x3], [-x4]])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [20, 4, -4, 9]

x_ponto, x_lista = penalidade("quadratica", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

# 8B

In [ ]:
# Defina as variáveis simbólicas
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')
xs_simbolicos = sp.Array([x1, x2, x3, x4])

# Matriz A e vetor b
A_simbolico = sp.Matrix([[-5, 7, -1, 1], [-1, -1, 1, 0], [0, 4, -6, 3], [0, 0, 0, 8], [-1, 1, 1, 0], [0, 2, 3, 0], [0, 1, -3, 3]])
b_simbolico = sp.Matrix([0, -1, -5, 3, 2, 0, 2])

# Função objetivo
ax_menos_b = A_simbolico * sp.Matrix(xs_simbolicos) - b_simbolico
objetivo_simbolico = 0.5 * ax_menos_b.dot(ax_menos_b)

objetivo = sp.lambdify(xs_simbolicos, objetivo_simbolico, "numpy")

# Restrições
h = sp.Matrix([x1 + x2 + x3 + x4 - 1])
g = sp.Matrix([[-x1], [-x2], [-x3], [-x4]])

# O ponto inicial deve violar as restrições na penalização externa.
x0 = [20, 4, -4, 9]

x_ponto, x_lista = penalidade("lagrangiana", objetivo_simbolico, xs_simbolicos, h, g, x0, 2)

In [ ]:
import sympy as sp

# Definindo as variáveis simbólicas
x1, x2, x3, s00, s10, s20, c = sp.symbols('x1 x2 x3 s00 s10 s20 c')

# Definindo a função de penalização objetivo
penalizacao_objetivo = c*((-x1 + s00**2)**2 + (-x2 + s10**2)**2 + (-x3 + s20**2)**2 + (x1 + x2 + x3 - 1)**2)/2 + 4.5*(-2*x1/3 + 1/6*x2 + x3)**2 + 0.5*(-x1 - 2*x2 + 4*x3 + 2)**2 + 0.5*(-x1 + 5*x2 + 3*x3 + 1)**2 + 4.5*(-1/6*x1 - x2 + x3/3 - 1)**2 + 0.5*(2*x1 - 3*x2 + x3 - 7)**2

# Calculando as derivadas parciais
d_penalizacao_objetivo_x1 = penalizacao_objetivo.diff(x1)
d_penalizacao_objetivo_x2 = penalizacao_objetivo.diff(x2)
d_penalizacao_objetivo_x3 = penalizacao_objetivo.diff(x3)
d_penalizacao_objetivo_s00 = penalizacao_objetivo.diff(s00)
d_penalizacao_objetivo_s10 = penalizacao_objetivo.diff(s10)
d_penalizacao_objetivo_s20 = penalizacao_objetivo.diff(s20)

print(d_penalizacao_objetivo_x1)
print(d_penalizacao_objetivo_x2)
print(d_penalizacao_objetivo_x3)
print(d_penalizacao_objetivo_s00)
print(d_penalizacao_objetivo_s10)
print(d_penalizacao_objetivo_s20)


c*(-2*s00**2 + 4*x1 + 2*x2 + 2*x3 - 2)/2 + 10.25*x1 - 8.5*x2 - 11.5*x3 - 15.5
c*(-2*s10**2 + 2*x1 + 4*x2 + 2*x3 - 2)/2 - 8.5*x1 + 47.25*x2 + 2.5*x3 + 31.0
c*(-2*s20**2 + 2*x1 + 2*x2 + 4*x3 - 2)/2 - 11.5*x1 + 2.5*x2 + 36.0*x3 + 1.0
2*c*s00*(s00**2 - x1)
2*c*s10*(s10**2 - x2)
2*c*s20*(s20**2 - x3)
